In [36]:
# ====================
# Importing Libraries 
# ====================
import pandas as pd 
import numpy as np 
from rdkit import Chem 
from rdkit.Chem import rdFingerprintGenerator

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import (roc_auc_score, accuracy_score, confusion_matrix, classification_report, precision_recall_curve, average_precision_score)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 

import matplotlib.pyplot as plt 
import joblib 
import seaborn as sn 
import random

# Set seeds for reproducibility 
seed = 67

random.seed(seed)
np.random.seed(seed)

# =================================
# Step 1: Load and process dataset 
# =================================
# Read dataset, skipping row 0 (metadata row)
df = pd.read_csv("excel data.csv", header=1, sep=',')

# Selecting relevant columns 
df = df[["Smiles", "Standard Type", "Standard Value", "Standard Units", "Target Name", "Assay Type"]]

# Removes rows missing essential values 
df = df.dropna(subset = ["Smiles", "Standard Value"])

# Converting IC50 values into nM 
def convert_to_nm(row):
    standard_value = float(row["Standard Value"])
    standard_unit = str(row["Standard Units"]).lower().strip()

    if "nm" in standard_unit:
        return standard_value
    elif "um" in standard_unit or "μm" in standard_unit:
        return standard_value * 1000
    elif "mm" in standard_unit:
        return standard_value * 1e06
    else:
        return None # Unrecognised unit to be dropped later
    
# Apply conversion function 
df["IC50"] = df.apply(convert_to_nm, axis=1)

# Drop any rows where unit conversion failed 
df = df.dropna(subset=["IC50"])
    
# Keep only PARP1 binding assays 
df = df[(df["Assay Type"] == "B") & (df["Target Name"] == "Poly [ADP-ribose] polymerase-1")]

# ================================
# Step 2: Assign Acitivity Labels
# ================================
def assign_activity_status(row):
    IC50 = float(row["IC50"])
    if IC50 <= 1000:
        return "Yes"
    else:
        return "No"
    
df["Active"] = df.apply(assign_activity_status, axis=1)

# ===================================================
# Step 3: Convert SMILES into molecular fingerprints 
# ===================================================
def smiles_to_fingerprint(smiles, n_bits=1024, radius=2):
    # Converts a SMILES string to a Morgan Fingerprint (binary vector). n_bits=fingerprint length, radius=ECFP radius 
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        print(f"Invalid SMILES: {smiles}")
        return None
    
    morgan_generator = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=n_bits)
    fp = morgan_generator.GetFingerprint(mol)
 
    # Converts RDKit vector into a NumPy array 
    arr = np.zeros((n_bits,), dtype=int)
    fp.ToBitString()
    for bit in fp.GetOnBits(): 
        arr[bit]=1 

    return arr

# generating fingerprints and labels 
fingerprints, labels = [], []

for idx, row in df.iterrows():
    fp = smiles_to_fingerprint(row["Smiles"])

    if fp is not None:
        fingerprints.append(fp)
        labels.append(1 if row["Active"] == "Yes" else 0)


# feature matrix:X and label vector:y
X = np.array(fingerprints, dtype=int)
y = np.array(labels, dtype=int) 

# =========================
# Step 4: Train/test/split
# =========================
# 80% training and 20% test. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


# =================================
# Step 5: Model comparison with CV 
# =================================
models = {"Logistic Regression": LogisticRegression(max_iter=2000, solver="saga"), "Random Forest":RandomForestClassifier(n_estimators=200, random_state=42), "SVM":SVC(kernel="rbf", probability=True, random_state=42)}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

print("Cross-validation results (ROC-AUC):")
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring="roc_auc", n_jobs=-1, verbose = 100)
    print(f"{name}: Mean ROC-AUC={scores.mean():.3f}plusminus {scores.std():.3f}")

# ===================================================================================================================================================================
# Step 6:Retraining random forest model with all available data to have as much info as possible and because it had highest roc-auc score
# ===================================================================================================================================================================
best_model = RandomForestClassifier(n_estimators=200, random_state=seed)
best_model.fit(X, y)

# =============================
# Step 7: Saving trained model 
# =============================
joblib.dump(best_model, "parp1_model.pkl")

# ======================================
# Step 8: Predicting with new compounds 
# ======================================

print("Do you want to predict on a new compound?(Y/N)")

new_compound = input()

if new_compound.lower() == "y":
    print("Input SMILES")
    new_smiles = input()
    fp = smiles_to_fingerprint(new_smiles)

    if fp is not None:
        prediction = best_model.predict([fp])[0]
        probability = best_model.predict_proba([fp])[0][1]
        print("Prediction:", "Active:" if prediction ==1 else "Inactive")
        print ("Probability of being Active:", probability)









Cross-validation results (ROC-AUC):
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   10.9s finished
Logistic Regression: Mean ROC-AUC=0.906plusminus 0.010
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.7s finished
Random Forest: Mean ROC-AUC=0.924plusminus 0.005
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    6.6s finished
SVM: Mean ROC-AUC=0.912plusminus 0.009
Do you want to predict on a new compound?(Y/N)
Input SMILES
Prediction: Active:
Probability of being Active: 0.7262121212121212
